In [2]:

import csv
import numpy as np
from scipy.stats import norm

file = open('bwght.csv')
file.close()

inmat = []
with open('bwght.csv') as file:
    ## Type what you want here and it works without closing the file
    reader = csv.reader(file)
    for line in reader:
        inmat.append(line)


inmat = [row[:4]+row[6:] for row in inmat]

print(inmat[0])
for ndx,item in enumerate(inmat[0]):
    print(ndx,item)
data = np.matrix(inmat[1:],dtype=float)



['faminc', 'cigtax', 'cigprice', 'bwght', 'parity', 'male', 'white', 'cigs', 'lbwght', 'bwghtlbs', 'packs', 'lfaminc']
0 faminc
1 cigtax
2 cigprice
3 bwght
4 parity
5 male
6 white
7 cigs
8 lbwght
9 bwghtlbs
10 packs
11 lfaminc


In [7]:
somkes=np.array(data[:,7]>0,dtype=int)
bigX = np.hstack((np.ones((n,1)),data[:,0],data[:,6]))
n = len(y)

NameError: name 'n' is not defined

In [12]:
n = len(y)
smokes=np.array(data[:,7]>0,dtype=int)
y=smokes
bigX = np.hstack((np.ones((n,1)),data[:,0],data[:,6]))


In [18]:
def bigF(val):
    return 1/(1+np.exp(-val))
bigF = lambda x: 1/(1+np.exp(-x))
littleF = lambda x:bigF(x)*(1-bigF(x))
def logL(b):
    return y*np.log(bigF(bigX.dot(b)))+(1-y)*np.log(1-bigF(bigX.dot(b)))
def deriv(b):
    suma = np.zero((r,))
    for i in range(n):
        xval = bigX[i,].reshape((r,))
        inside = xval.T.dot(b)
        suma += (y[i]-bigF(inside))[0,0]*(xval)
    return suma
def hess(b):
    suma = np.zero((r,))
    for i in range(n):
        xval = bigX[i,].reshape((r,))
        inside = xval.T.dot(b)
        suma += -littleF(inside).dot(xval(xval.T))
    return suma

In [ ]:
from scipy.optimize import root
b = root(deriv,x0=np.zeros((r,)),jac=hess)

In [ ]:
hess_star = b.fjac
b = b.x

In [22]:
def cross(*args):  # Can put either 1 or 2 things into this function
    x = args[0]
    if len(args)==1:
        y = x
    else:
        y = args[1]
    return(x.T.dot(y))
def least_squares(X,y):
    return(np.linalg.solve(cross(X),cross(X,y)))
def ols(X,y):
    # goals: bhats, se, tstats, pvals, Rsq
    b = least_squares(X,y)
    yhat = X.dot(b)
    e = y-yhat
    RSS = cross(e)[0,0]
    ybar = y.mean()
    e_0 = y-ybar
    TSS = cross(e_0)[0,0]
    Rsq = 1-RSS/TSS
    sig = np.sqrt(RSS/n)
    vb = np.linalg.inv(cross(X))*(sig**2)
    se = np.sqrt(np.diag(vb)).reshape(-1,1)
    tstats = b/se
    pval = norm.cdf(-np.abs(tstats))*2
    return(b,se,tstats,pval,Rsq)
def white(X,y):
    # goals: bhats, se, tstats, pvals, Rsq
    b = least_squares(X,y)
    yhat = X.dot(b)
    e = y-yhat
    RSS = cross(e)[0,0]
    ybar = y.mean()
    e_0 = y-ybar
    TSS = cross(e_0)[0,0]
    Rsq = 1-RSS/TSS
    sig = np.sqrt(RSS/n)
    vb = np.diagflat(np.power(e,2))
    vb = X.T.dot(vb).dot(X)
    vb = np.linalg.inv(cross(X))*vb*np.linalg.inv(cross(X))
    se = np.sqrt(np.diag(vb)).reshape(-1,1)
    tstats = b/se
    pval = norm.cdf(-np.abs(tstats))*2
    return(b,se,tstats,pval,Rsq)
def tidyols(X,y):
    (b,se,tstats,pval,Rsq) = ols(X,y)
    print(np.hstack((b,se,tstats,pval)))
    print("RSq="+str(Rsq))    
def tidywhitey(X,y):
    (b,se,tstats,pval,Rsq) = white(X,y)
    print(np.hstack((b,se,tstats,pval)))
    print("RSq="+str(Rsq))    

In [14]:
y = data[:,3]
y
x = data[:,7]
x
n = len(y)
bigX = np.ones((n,1))
bigX = np.hstack((bigX,x))
bigX

bigX = np.hstack((np.ones((n,1)),data[:,7],data[:,0],data[:,5],data[:,6]))
    
tidyols(bigX,y)
tidywhitey(bigX,y)


[[  1.12065256e+02   1.44005920e+00   7.78198954e+01   0.00000000e+00]
 [ -4.74159926e-01   9.07534706e-02  -5.22470296e+00   1.74434907e-07]
 [  6.00548455e-02   3.03915114e-02   1.97604011e+00   4.81502344e-02]
 [  3.14523963e+00   1.06882961e+00   2.94269509e+00   3.25368704e-03]
 [  5.40726154e+00   1.36284948e+00   3.96761463e+00   7.25955979e-05]]
RSq=0.046451096834
[[  1.12065256e+02   1.44906731e+00   7.73361287e+01   0.00000000e+00]
 [ -4.74159926e-01   8.92811029e-02  -5.31086547e+00   1.09105852e-07]
 [  6.00548455e-02   2.96038813e-02   2.02861391e+00   4.24976302e-02]
 [  3.14523963e+00   1.06335018e+00   2.95785875e+00   3.09784038e-03]
 [  5.40726154e+00   1.42936280e+00   3.78298745e+00   1.54957230e-04]]
RSq=0.046451096834


Here is a linear regression line: $y_i=\beta_0+\beta_1x_i+e_i$


In [23]:
n = 100
r = 2
sigx=1
sig=1
x= np.random.normal(size=(n,r),loc=0,scale=sigx)
y= np.random.normal(size=(n,1),loc=0,scale=sig)
bigX = np.hstack((np.ones((n,1)),x))
ols(bigX,y)[-1]

0.017858974247728088

In [24]:
n = 100
r = 2
sigx=1
sig=1
rho = o
rsq=np.zeros((1000,))
y= np.random.normal(size=(n,1),loc=0,scale=sig)
for i in range(1,n):
    y[i] += rho*y[i-1]
y

NameError: name 'rho' is not defined

In [ ]:
r=2
n=50
outp1 = np.zeros((1000,2))
for isim in range(1000):
    x = np.random.normal((50,1))
    e = np.random.normal((50,1))
    y = 0.5 + x + e
    y = (y>=0).astype(int)
    bigx = np.hstack((np.ones((50,1)),x))
    
    b = root(deriv,x0=np.zeros((r,)),jac=hess)
    outp1[isim,:]=b,x
    outp2[isim,:]=np.